In [1]:
import pandas as pd
import numpy as np

In [2]:
data_path = "/content/drive/MyDrive/DS102/DoAn/data/"
NA = data_path + "NAnh_TrueVocab.csv"
QA = data_path + "QuocAnh_TrueVocab.csv"
DT = data_path + "tueje_vocab.csv"

In [6]:
NA_df = pd.read_csv(NA)
QA_df = pd.read_csv(QA)
DT_df = pd.read_csv(DT)

In [7]:
QA_df.head()

,word,label
0,room,3
1,staff,2
2,hotel,6
3,location,1
4,breakfast,5


In [8]:
def processLabel(labels):
  if len(labels) == 1:
    return [int(labels)]
  else:
    return [int(label) for label in labels.replace(" ", "").split(",")]

dfs = [NA_df, QA_df, DT_df]
labels = []
for i, df in enumerate(dfs):
  label = df["label"].apply(lambda x: processLabel(x)).to_list()
  labels.append(label)

In [9]:
labels[0][14] + labels[1][14] + labels[2][14]

[1, 2, 6, 2]

# IAA with fleiss kappa

In [20]:
label_num = 6
labels_vec = []
for i in range(len(labels[0])):
  vec = [0 for _ in range(label_num)]
  for j in range(len(labels)):
    for label in labels[j][i]:
      vec[label-1] += 1
  labels_vec.append(vec)

In [28]:
labels_vec = np.array(labels_vec)

In [56]:
def fleiss_kappa(labels):
  Pj = np.sum(labels_vec, axis=0) / (np.sum(labels_vec))
  Pe = np.sum(Pj ** 2)

  Pi = 0
  for word in labels:
    Pi += (np.sum(word ** 2) - np.sum(word)) / (np.sum(word) * (np.sum(word)-1))
  P = Pi / labels.shape[0]
  k = (P - Pe) / (1- Pe)
  return k

In [57]:
fleiss_kappa(labels_vec)

0.5505094255127742

# True labels

In [ ]:
true_labels = []
for i in range(len(labels[0])):
  l = []
  for arr in labels:
    l += arr[i]
  unique_labels, counts = np.unique(l, return_counts=True)
  true_labels.append(unique_labels[np.argmax(counts)])
true_labels = np.array(true_labels)

In [ ]:
np.unique(true_labels, return_counts=True)

(array([1, 2, 3, 4, 5, 6]), array([192, 187, 170,  70,  70, 312]))

In [ ]:
words = QA_df["word"].to_numpy()
ids = np.nonzero()
hotel_words = words[ids]
hotel_words.shape

(689,)

In [ ]:
hotel_labels = true_labels[true_labels != 6]
hotel_labels.shape

(689,)

In [ ]:
np.array([hotel_words, hotel_labels])

(2, 689)

In [ ]:
hotel_labels_df = pd.DataFrame(np.array([hotel_words, hotel_labels]).T, columns=["word", "label"])
hotel_labels_df.head()

,word,label
0,room,3
1,staff,2
2,hotel,1
3,location,1
4,breakfast,5


In [ ]:
hotel_labels_path = data_path + "true_labels.csv"
hotel_labels_df.to_csv(hotel_labels_path, index=False)